In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


To install dependencies

Note: After installation, please restart the runtime if it is required.

In [2]:
!pip install ultralytics
!pip install av
!pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=4017d26ca581b45984d05c3166d61d77814cecbf06da2dcd2ead06305b554b76
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy


The Player & ball tracking with norfair

In [3]:
import sys
root_path = '/content/drive/MyDrive/basketball-broadcaster'
sys.path.append(root_path)

import os
os.chdir(root_path)

import av
import time
from PIL import Image
import numpy as np

from pb_detector import PBDetector
from pb_tracker import PBTracker

input_ = av.open("videos/test1_clip.mp4")
output = av.open("output.mp4", "w")

detector = PBDetector()
tracker = PBTracker()

# Make an output video stream using the input video stream as a template
in_video_stream = input_.streams.video[0]
out_video_stream = output.add_stream(codec_name="h264",
                                      width=in_video_stream.width,
                                      height=in_video_stream.height,
                                      rate=int(in_video_stream.average_rate),
                                      bit_rate=in_video_stream.bit_rate,
                                      pix_fmt=in_video_stream.pix_fmt)

# Make an output audio stream using the input audio stream as a template
in_audio_stream = input_.streams.audio[0]
out_audio_stream = output.add_stream(template=in_audio_stream)

frame_idx = 0
det_cycle = 3

st = time.time()
prec_time = st
decoding_time = 0
inference_time = 0
mot_time = 0
encoder_time = 0

for packet in input_.demux():
    # We need to skip the "flushing" packets that `demux` generates.
    if packet.dts is None:
        continue

    if packet.stream == in_video_stream:
        for frame in packet.decode():
            # To get an input frame
            image = frame.to_image() # PIL image
            image = np.array(image)
            current_time = time.time()
            decoding_time += (current_time - prec_time)
            prec_time = current_time

            ##############  To process by AI engine  ############

            # To process the input frame
            player_detections = []
            ball_detections = []
            if frame_idx % det_cycle == 0:
                player_detections, ball_detections = detector.detect(image)
                current_time = time.time()
                inference_time += (current_time - prec_time)
                prec_time = current_time

            player_detections, ball_detections = tracker.run(image, player_detections, ball_detections)
            frame_idx += 1

            # To draw the result
            image = Image.fromarray(image)
            image = tracker.draw_detections(image, player_detections, ball_detections)
            current_time = time.time()
            mot_time += (current_time - prec_time)
            prec_time = current_time

            ####################################################

            output_frame = av.VideoFrame.from_image(image)
            output_frame.pts = None
            for output_packet in out_video_stream.encode(output_frame):
                # Write the packets to the container
                output_packet.stream = out_video_stream
                output.mux(output_packet)
    elif packet.stream == in_audio_stream:
        packet.stream = out_audio_stream
        output.mux(packet)
        current_time = time.time()
        encoder_time += (current_time - prec_time)
        prec_time = current_time

elapsed = time.time() - st

input_.close()
output.close()


0: 384x640 12 persons, 202.2ms
Speed: 26.2ms preprocess, 202.2ms inference, 48.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 7.8ms
Speed: 3.1ms preprocess, 7.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 13.7ms
Speed: 3.8ms preprocess, 13.7ms inference, 8.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 14.2ms
Speed: 5.4ms preprocess, 14.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 19.0ms
Speed: 3.1ms preprocess, 19.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 10.5ms
Speed: 6.8ms preprocess, 10.5ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 6.2ms
Speed: 2.7ms preprocess, 6.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 1 sports ball, 7.5ms
Speed: 3.0ms preprocess, 7.5ms inference, 1.7ms postprocess

In [4]:
# To print the FPS
print('Processing FPS: ', int(in_video_stream.frames / elapsed))
print('Decoding time: ', int(decoding_time))
print('Inference time: ', int(inference_time))
print('MOT time: ', int(mot_time))
print('Encoder time: ', int(encoder_time))

Processing FPS:  4
Decoding time:  124
Inference time:  31
MOT time:  40
Encoder time:  207
